In [37]:
import torch
from lightning import LightningModule, Trainer
from torch import nn, Tensor
from torch_geometric.data import Data, Batch
from torch_geometric.nn import SAGEConv, global_mean_pool
from torchmetrics import Accuracy, MeanMetric
import torch.nn.functional as F
from lightning import LightningDataModule
from torch.utils.data import random_split, DataLoader, Dataset
from torchvision.transforms import transforms
from torchvision.datasets import Imagenette

In [38]:
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

class Subset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)


class ImagenetteDataModule(LightningDataModule):
    def __init__(self, data_dir: str = '../data/', batch_size: int = 64):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

    def prepare_data(self) -> None:
        Imagenette(root=self.data_dir, split='train', download=True)
        Imagenette(root=self.data_dir, split='val', download=True)

    def setup(self, stage: str = None):
        full = Imagenette(root=self.data_dir, split='train')
        train_data, val_data = random_split(
            full, [8500, 969], generator=torch.Generator().manual_seed(42)
        )
        self.train_data = Subset(train_data, transform=train_transform)
        self.val_data = Subset(val_data, transform=val_transform)
        self.test_data = Imagenette(root=self.data_dir, split='val', transform=val_transform)

    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=self.batch_size, num_workers=4, pin_memory=True)

    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size=self.batch_size, num_workers=4, pin_memory=True)

    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size=self.batch_size, num_workers=4, pin_memory=True)

In [39]:
class PatchExtractor(nn.Module):
    def __init__(self, in_dim=3, out_dim=256):
        super().__init__()
        self.out_dim = out_dim
        self.convs = nn.Sequential(
            nn.Conv2d(in_dim, out_dim // 8, 3, stride=2, padding=1),
            nn.BatchNorm2d(out_dim // 8),
            nn.ReLU(),
            nn.Conv2d(out_dim // 8, out_dim // 4, 3, stride=2, padding=1),
            nn.BatchNorm2d(out_dim // 4),
            nn.ReLU(),
            nn.Conv2d(out_dim // 4, out_dim // 2, 3, stride=2, padding=1),
            nn.BatchNorm2d(out_dim // 2),
            nn.ReLU(),
            nn.Conv2d(out_dim // 2, out_dim, 3, stride=2, padding=1),
            nn.BatchNorm2d(out_dim),
            nn.ReLU(),
            nn.Conv2d(out_dim, out_dim, 3, stride=1, padding=1),
            nn.BatchNorm2d(out_dim),
        )

    def forward(self, x):
        return self.convs(x).flatten(2).transpose(2, 1)


In [40]:
def generate_neighbour_edge_index(n: int):
    edge_index = []
    for i in range(n):
        for j in range(n):
            index = i * n + j
            if i > 0:
                edge_index.append([index, (i - 1) * n + j])
            if i < n - 1:
                edge_index.append([index, (i + 1) * n + j])
            if j > 0:
                edge_index.append([index, i * n + (j - 1)])
            if j < n - 1:
                edge_index.append([index, i * n + (j + 1)])
    return torch.tensor(edge_index, dtype=torch.long).T.contiguous()

In [41]:
def patches_to_graph(x: Tensor, edge_index: Tensor = None):
    batch_size = x.size()[0]
    data_list = [
        Data(x=x[index], edge_index=edge_index) for index in range(batch_size)
    ]
    return Batch.from_data_list(data_list)

In [42]:
class VisionGNN(LightningModule):
    def __init__(self):
        super().__init__()
        self.save_hyperparameters()
        self.patch_extractor = PatchExtractor(3, 256)
        self.conv1 = SAGEConv(256, 128)
        self.conv2 = SAGEConv(128, 64)
        self.lin1 = nn.Linear(64, 32)
        self.lin2 = nn.Linear(32, 10)
        self.criterion = nn.CrossEntropyLoss()
        self.train_acc = Accuracy(task="multiclass", num_classes=10)
        self.val_acc = Accuracy(task="multiclass", num_classes=10)
        self.test_acc = Accuracy(task="multiclass", num_classes=10)
        self.train_loss = MeanMetric()
        self.val_loss = MeanMetric()
        self.test_loss = MeanMetric()

    def setup(self, stage):
        self.edge_index = generate_neighbour_edge_index(14).to(self.device)

    def model_step(self, batch):
        x, y = batch
        logits = self.forward(x)
        loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        return loss, preds, y

    def forward(self, x):
        x = self.patch_extractor(x)
        graph_batch = patches_to_graph(x, self.edge_index)
        x, edge_index, batch = graph_batch.x, graph_batch.edge_index, graph_batch.batch
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        x = self.lin1(x)
        x = F.relu(x)
        x = self.lin2(x)
        return x

    def on_train_start(self) -> None:
        self.val_loss.reset()
        self.val_acc.reset()

    def training_step(self, batch):
        loss, preds, targets = self.model_step(batch)

        self.train_loss(loss)
        self.train_acc(preds, targets)
        self.log("train/loss", self.train_loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train/acc", self.train_acc, on_step=False, on_epoch=True, prog_bar=True)

        return loss

    def validation_step(self, batch):
        loss, preds, targets = self.model_step(batch)

        self.val_loss(loss)
        self.val_acc(preds, targets)
        self.log("val/loss", self.val_loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val/acc", self.val_acc, on_step=False, on_epoch=True, prog_bar=True)

    def test_step(self, batch):
        loss, preds, targets = self.model_step(batch)

        self.test_loss(loss)
        self.test_acc(preds, targets)
        self.log("test/loss", self.test_loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test/acc", self.test_acc, on_step=False, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=0.0003)
        return optimizer


In [43]:
model = VisionGNN()
datamodule = ImagenetteDataModule(batch_size=64)
trainer = Trainer(max_epochs=100)
trainer.fit(model=model, datamodule=datamodule)
trainer.test(model=model, datamodule=datamodule, ckpt_path='best')

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name            | Type               | Params
--------------------------------------------------------
0  | patch_extractor | PatchExtractor     | 979 K 
1  | conv1           | SAGEConv           | 65.7 K
2  | conv2           | SAGEConv           | 16.4 K
3  | lin1            | Linear             | 2.1 K 
4  | lin2            | Linear             | 330   
5  | criterion       | CrossEntropyLoss   | 0     
6  | train_acc       | MulticlassAccuracy | 0     
7  | val_acc         | MulticlassAccuracy | 0     
8  | test_acc        | MulticlassAccuracy | 0     
9  | train_loss      | MeanMetric         | 0     
10 | val_loss        | MeanMetric         | 0     
11 | test_loss       | MeanMetric         | 0     
--------------------------------------------------------
1.1 M     Trainable params
0

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
Restoring states from the checkpoint path at /home/bwojcik/PycharmProjects/vision_gnn/notebooks/lightning_logs/version_4/checkpoints/epoch=99-step=13300.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /home/bwojcik/PycharmProjects/vision_gnn/notebooks/lightning_logs/version_4/checkpoints/epoch=99-step=13300.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/acc            0.8341401219367981
        test/loss           0.5547029972076416
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test/loss': 0.5547029972076416, 'test/acc': 0.8341401219367981}]